In [6]:
import pandas as pd

from oof_unet.data import OofUniqueMachineDataset, VolumeBasedBatchSampler
from torch.utils.data import DataLoader


In [7]:
df = pd.read_csv('../oof_unet/Dataset001_final.csv')
df_train = df[df['split'] == 'Train']
df_train = df_train.reset_index(drop=True)

list_train = df_train['id_dosemap'].tolist()


In [8]:
from sklearn.model_selection import KFold

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
for fold, (_, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold'] = f'Val_{fold}'

In [13]:
df_train.to_csv('Dataset001_final_train.csv', index=False)

In [12]:
df_test = df[df['split'] == 'Test']
df_test = df_test.reset_index(drop=True)

df_test.to_csv('Dataset001_final_test.csv', index=False)

In [16]:
train_dataset = OofUniqueMachineDataset(list_dosemaps=list_train, split='Train')
train_sampler = VolumeBasedBatchSampler(dataset=train_dataset, n_units=20, unit_volume=10_000_000)

dataloader = DataLoader(train_dataset, batch_sampler=train_sampler, num_workers=4)


In [17]:
i = 0

for batch in dataloader:

    print(batch["in_field"].shape)

    i += 1

    if i == 10:
        break

torch.Size([5, 240, 162, 876])
torch.Size([5, 240, 162, 876])
torch.Size([5, 240, 162, 876])
torch.Size([4, 220, 233, 787])
torch.Size([4, 220, 233, 787])
torch.Size([4, 220, 233, 787])
torch.Size([4, 220, 233, 787])
torch.Size([4, 220, 233, 787])
torch.Size([2, 220, 233, 787])
torch.Size([10, 194, 139, 705])


In [14]:
batch["in_field"].shape

torch.Size([8, 172, 209, 623])